In [19]:
import requests
from bs4 import BeautifulSoup
import urllib
import itertools
import os
import re
import functools
from multiprocessing.dummy import Pool as ThreadPool

In [20]:
base_url = "http://gd2.mlb.com/components/game/mlb/"

In [21]:
date_tuples = [(6,19,2019)] # put the dates to download pitchfx, as tuples (month, day, year)

In [22]:
# This builds a list of games url paths to download
games = []
for date in date_tuples:
    month, day, year = date
    try:
        day_url = base_url+f"year_{year}/month_{str(month).zfill(2)}/day_{str(day).zfill(2)}"
        soup = BeautifulSoup(requests.get(day_url).text)
        games += [ f"{day_url}/{node.get('href').split('/')[1]}" for node in soup.find_all('a') if 'gid' in node.get('href')]
    except:
        pass

In [23]:
games # just a check to see if there's data here

['http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_anamlb_tormlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_balmlb_oakmlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_bosmlb_minmlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_chamlb_chnmlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_clemlb_texmlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_colmlb_arimlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_detmlb_pitmlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_houmlb_cinmlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_kcamlb_seamlb_1',
 'http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_miamlb_slnmlb_1',


In [24]:
# this code downloads the pitchfx xml for the games discovered on the date
# it uses multi-threading to download faster(a lot)
pool = ThreadPool(4)
results = pool.map(lambda x: urllib.request.urlretrieve(f"{x}/inning/inning_all.xml", f"{x.split('/')[9]}.xml") if not os.path.isfile(f"{x.split('/')[9]}.xml") else None, games)
pool.close()
pool.join()

## Code below is meant to parse the xml files downloaded above and put into a pandas dataframe and also get the homeplate ump

In [25]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os
from scipy.spatial import distance
from scipy.spatial.distance import cdist
pd.set_option('display.max_columns', 500)

In [26]:
# this is later used to find the center of the strike zone
def to_inches(height_string):
    feet, inches = re.findall(re.compile("(\d+)['] ?(\d\d?)"),height_string)[0]
    return int(feet) * 12 + int(inches)

In [27]:
# well use this later to make the highlights easier to get to
pd.set_option("display.max_rows",300)
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)


In [28]:
def get_ump(game):
    urllib.request.urlretrieve(f"{game}/players.xml","players.xml")
    with open("players.xml", 'r') as file:
        root = ET.parse(file).getroot()
    for ump in root.iter(tag="umpire"):
        if ump.attrib["position"] == "home":
            return ump.attrib["name"]

In [29]:
get_ump(games[1]) # test if get_ump works

'Mike Estabrook'

In [30]:
# builds a list for each game
at_bats = []
for game in games:
    print(game)
    with open(f"{game.split('/')[9]}.xml", 'r') as file:
        root = ET.parse(file).getroot()
    home_ump = get_ump(game)
    for atbat in root.iter(tag='atbat'):
        for pitch in atbat.iter(tag='pitch'):
            data = pitch.attrib
            data["home_ump"] = home_ump
            data["batter"] = atbat.attrib["batter"]
            data["pitcher"] = atbat.attrib["pitcher"]
            data["atbat_result"] = atbat.attrib["event"]
            data["atbat_des"] = atbat.attrib["des"]
            data["atbat_num"] = atbat.attrib["num"]
            data["atbat_b_height_inches"] = to_inches(atbat.attrib["b_height"])
            data["game_id"] = f"{game.split('/')[9]}"
            at_bats.append(data)

http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_anamlb_tormlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_balmlb_oakmlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_bosmlb_minmlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_chamlb_chnmlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_clemlb_texmlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_colmlb_arimlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_detmlb_pitmlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_houmlb_cinmlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_kcamlb_seamlb_1
http://gd2.mlb.com/components/game/mlb/year_2019/month_06/day_19/gid_2019_06_19_miamlb_slnmlb_1
http://gd2.mlb.com/components/game/mlb/y

## umpire closeness starts here

In [31]:
df_ab = pd.DataFrame(at_bats)

In [32]:
df_ab.dropna(inplace=True)
df_ab["player_z_adjust"] = df_ab["atbat_b_height_inches"] / 24.0
df_ab[["px","pz"]] = df_ab[["px","pz"]].apply(pd.to_numeric)
df_ab["x_from_origin"] = df_ab.px
df_ab["z_from_origin"] = (df_ab.player_z_adjust * .80) - df_ab.pz
df_ab["distance_from_center"] = df_ab.apply(lambda row: distance.euclidean((row.px * 1.1,row.pz),(0,row.player_z_adjust * .82)), axis=1)
df_ab["simple_pitch"] = df_ab.pitch_type.map({
    'SI': 'fastball', 'FC': 'fastball', 'CH' : 'changeup', 'FF': 'fastball', 'CU': 'breaking', 'SL': 'breaking', 'FS': 'fastball', 'FT': 'fastball', 'KC': 'breaking'
})
df_ab["url"] = df_ab.play_guid.apply(lambda x: f"https://baseballsavant.mlb.com/sporty-videos?playId={x}")


In [33]:
called_strikes = df_ab.query("des == 'Called Strike'").copy()
called_balls = df_ab.query("des == 'Ball'").copy()

In [ ]:
# this code will list through all the umpires and put their top pairs of different calls based on how alike they were. both break length and break angle are taken into account for likeness

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

# this is where the magic happens
for ump in df_ab["home_ump"].unique():
    print(ump)
    df1 = called_strikes.query(f'home_ump == "{ump}"').copy()
    df2 = called_balls.query(f'home_ump == "{ump}"').copy()

    df1['point'] = [(x, y, a,b) for x,y,a,b in zip(df1['px'] * 10, df1['pz'] * 10,df1['break_length'].astype(float),df1['break_angle'].astype(float)/10)]
    df2['point'] = [(x, y, a,b) for x,y,a,b in zip(df2['px'] * 10, df2['pz'] * 10,df2['break_length'].astype(float),df2['break_angle'].astype(float)/10)]

    df2['closest'] = [closest_point(x, list(df1['point'])) for x in df2['point']]
    df2['close_guid'] = [match_value(df1, 'point', x, 'play_guid') for x in df2['closest']]
    df2['close_url'] = df2.close_guid.apply(lambda x: f"https://baseballsavant.mlb.com/sporty-videos?playId={x}")
    df2['close_game'] = [match_value(df1, 'point', x, 'game_id') for x in df2['closest']]
    df2['close_simple_pitch'] = [match_value(df1, 'point', x, 'simple_pitch') for x in df2['closest']]
    df2['close_break_length'] = [match_value(df1, 'point', x, 'break_length') for x in df2['closest']]
    df2['close_break_angle'] = [match_value(df1, 'point', x, 'break_angle') for x in df2['closest']]
    
    df2["distance_to_strike"] = df2.apply(lambda row: distance.euclidean(row.point,row.closest), axis=1)
    display(df2[df2.game_id == df2.close_game]["simple_pitch,close_simple_pitch,break_length,close_break_length,game_id,close_game,url,close_url,distance_to_strike".split(",")].sort_values(["game_id","distance_to_strike"]).head(20).style.format({'url': make_clickable,'close_url': make_clickable}))


Mike Winters


,simple_pitch,close_simple_pitch,break_length,close_break_length,game_id,close_game,url,close_url,distance_to_strike
4,fastball,fastball,4.8,3.6,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=2c1fa3f0-1b42-4c28-874f-c3c83ad182a6,https://baseballsavant.mlb.com/sporty-videos?playId=26f21fdf-7750-4a1b-8676-9daca31e6a1c,1.38578
75,fastball,fastball,4.8,6.0,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=d91d921e-f114-4af9-8839-ecb1ac4dc01b,https://baseballsavant.mlb.com/sporty-videos?playId=1c2857cd-20c1-4082-93c6-041306d30b93,1.42632
68,fastball,fastball,4.8,6.0,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=21d56fee-f4d4-4c86-a95f-ce69f521308f,https://baseballsavant.mlb.com/sporty-videos?playId=d545bd59-2848-4b80-94e0-46b3337266c7,1.67332
158,breaking,changeup,9.6,9.6,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=2a8d3c05-0d76-4be3-ac58-037add2e8684,https://baseballsavant.mlb.com/sporty-videos?playId=e3f87ca2-a880-415b-a156-414194cc9d6c,1.70282
110,breaking,breaking,10.8,10.8,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=8dba623d-1d30-438e-aba2-6d8ef8e5fc76,https://baseballsavant.mlb.com/sporty-videos?playId=4a1ae13c-2b75-4f35-a66b-02f78d03d644,1.70294
268,fastball,fastball,4.8,4.8,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=5c4db0ef-e46c-4a1c-b27d-2d15fadd392b,https://baseballsavant.mlb.com/sporty-videos?playId=2a7a6f08-e040-448e-9335-3f8030e36523,1.73494
240,fastball,fastball,4.8,4.8,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=336dc0ca-ed04-4aee-87bf-4562bd59cffe,https://baseballsavant.mlb.com/sporty-videos?playId=cf808118-3e46-45b0-b9c4-15176de0903d,1.80544
10,changeup,fastball,7.2,6.0,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=e6f37d1f-d99b-4434-8cf1-b0390faf8c0e,https://baseballsavant.mlb.com/sporty-videos?playId=d545bd59-2848-4b80-94e0-46b3337266c7,1.89621
148,fastball,fastball,4.8,4.8,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=c8ec12f3-35d3-48f1-870e-cd2efe5b5697,https://baseballsavant.mlb.com/sporty-videos?playId=3fa7ad37-081b-4d3e-a994-a477bbf2ec60,1.91426
33,fastball,fastball,4.8,4.8,gid_2019_06_19_anamlb_tormlb_1,gid_2019_06_19_anamlb_tormlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=f0aa0c24-3258-40ae-850a-2a5066c2c673,https://baseballsavant.mlb.com/sporty-videos?playId=fdf3f8f0-f6a9-43ac-ad8c-144d6f1f1931,2.31206


Mike Estabrook


,simple_pitch,close_simple_pitch,break_length,close_break_length,game_id,close_game,url,close_url,distance_to_strike
581,fastball,fastball,3.6,3.6,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=ec653438-5f4a-4456-a4bd-8e5a2ef1d31d,https://baseballsavant.mlb.com/sporty-videos?playId=ec509c17-19b8-4f4c-ac51-8bcd6dfc9570,1.81813
574,fastball,fastball,4.8,4.8,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=09649e48-a2a5-454f-86dc-5edd84b7abaa,https://baseballsavant.mlb.com/sporty-videos?playId=67333235-c002-4006-9e1d-68edf301c429,2.196
506,changeup,fastball,7.2,6.0,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=68cccea3-87a7-463e-b8ad-c8fc3f0bebfa,https://baseballsavant.mlb.com/sporty-videos?playId=a38ffebb-3748-4e1c-82ef-6f353d4f55c7,2.20762
572,fastball,fastball,4.8,6.0,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=01448f74-9ca9-419c-89f5-ccca39a908e4,https://baseballsavant.mlb.com/sporty-videos?playId=a38ffebb-3748-4e1c-82ef-6f353d4f55c7,2.23437
554,fastball,fastball,4.8,3.6,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=2a1eccb2-f0a4-4cbb-a385-2de748107ba9,https://baseballsavant.mlb.com/sporty-videos?playId=831e045d-2a0c-4ea6-a0bc-c5eec736124c,2.76615
434,changeup,fastball,7.2,6.0,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=4e3ccffc-6db4-4cd7-8c7d-412db0310c9c,https://baseballsavant.mlb.com/sporty-videos?playId=a38ffebb-3748-4e1c-82ef-6f353d4f55c7,2.80179
499,changeup,breaking,7.2,9.6,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=e87d32da-a46e-4ebf-bfe0-ccf53bdbc38b,https://baseballsavant.mlb.com/sporty-videos?playId=cf5b916c-80c1-4f46-8b99-568186136dcd,2.8859
627,fastball,fastball,3.6,3.6,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=2815423a-50c7-455b-b412-905540d0fe43,https://baseballsavant.mlb.com/sporty-videos?playId=1e351d57-d5aa-4463-8bc2-2216981c7bc6,3.07415
387,fastball,fastball,4.8,4.8,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=0211f845-8904-44cf-87a0-3a6143d417ea,https://baseballsavant.mlb.com/sporty-videos?playId=98c8353f-489f-4e08-b733-517b20dd817b,3.14541
361,fastball,changeup,4.8,7.2,gid_2019_06_19_balmlb_oakmlb_1,gid_2019_06_19_balmlb_oakmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=fddf7581-3ec6-4f67-a5ad-33224f37e0ae,https://baseballsavant.mlb.com/sporty-videos?playId=4e6f120e-56e8-4c9a-9a69-3b194a4ea697,3.21521


Jim Reynolds


...

,simple_pitch,close_simple_pitch,break_length,close_break_length,game_id,close_game,url,close_url,distance_to_strike
1043,fastball,fastball,6.0,6.0,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=d727deb6-5c18-4a80-8351-f4af2bd65d94,https://baseballsavant.mlb.com/sporty-videos?playId=f652bf56-90db-4b5f-b551-772a10a155a4,1.68808
1020,changeup,fastball,6.0,4.8,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=66ee23f2-7ccc-4604-9719-0fb574ea5978,https://baseballsavant.mlb.com/sporty-videos?playId=64374069-4fa4-477e-a3ce-04df4bad9124,1.81395
1013,changeup,fastball,6.0,4.8,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=6c145247-ff83-4e21-b0c8-a086d86aafe9,https://baseballsavant.mlb.com/sporty-videos?playId=64374069-4fa4-477e-a3ce-04df4bad9124,1.91008
1054,fastball,fastball,6.0,4.8,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=4ced5634-60c0-491d-91e6-3109ade59a56,https://baseballsavant.mlb.com/sporty-videos?playId=010d4aff-14ef-4de9-b6cc-39aed8fdc325,1.99088
1215,fastball,changeup,4.8,6.0,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=16412869-666b-45b8-a3cc-188d3d38a9c6,https://baseballsavant.mlb.com/sporty-videos?playId=218e282e-b16b-40d6-a34b-0e5b846450be,2.06
1275,breaking,fastball,6.0,4.8,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=fc0bd182-8e19-4622-9fc6-92d0b9c44726,https://baseballsavant.mlb.com/sporty-videos?playId=64374069-4fa4-477e-a3ce-04df4bad9124,2.18257
988,fastball,fastball,3.6,3.6,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=30338020-b133-4756-9830-e3b9267d6aef,https://baseballsavant.mlb.com/sporty-videos?playId=13db382e-24bb-412e-9f41-bebb23436b18,2.22926
1016,changeup,fastball,6.0,4.8,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=5475e081-0ccf-43c5-a8c9-094a00f558b3,https://baseballsavant.mlb.com/sporty-videos?playId=64374069-4fa4-477e-a3ce-04df4bad9124,2.26883
1036,fastball,fastball,3.6,3.6,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=0a36a227-6615-4928-8436-ac32811f3e60,https://baseballsavant.mlb.com/sporty-videos?playId=92e760a4-c7c1-4cb4-a195-171617f6061a,2.4779
1229,fastball,fastball,6.0,7.2,gid_2019_06_19_chamlb_chnmlb_1,gid_2019_06_19_chamlb_chnmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=84be26bb-8dab-415d-bdce-73a462f17346,https://baseballsavant.mlb.com/sporty-videos?playId=13ac1ab8-3317-49ef-8632-1973891440f2,2.63522


Bill Miller


,simple_pitch,close_simple_pitch,break_length,close_break_length,game_id,close_game,url,close_url,distance_to_strike
1557,fastball,fastball,4.8,4.8,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=412e6f74-bd02-4381-aad5-d7f468853b96,https://baseballsavant.mlb.com/sporty-videos?playId=ec6d661e-fa62-4bee-8722-457c536d57a7,1.74402
1382,fastball,fastball,2.4,3.6,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=d78aa720-2d34-4fb2-97f5-f7b559301224,https://baseballsavant.mlb.com/sporty-videos?playId=150068b7-ac14-42a6-bea5-c2c87c5885b7,1.87254
1575,changeup,breaking,7.2,8.4,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=af9fd0f7-9239-49b5-b497-ef2f70ec2550,https://baseballsavant.mlb.com/sporty-videos?playId=f9198c79-7049-4e06-8320-bbd8895d225d,2.13129
1444,fastball,fastball,3.6,3.6,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=5e450822-a567-44c2-b3e1-6ff925f8fa49,https://baseballsavant.mlb.com/sporty-videos?playId=c7ee448b-94ff-4926-8c59-59dbd4d93a81,2.38126
1528,fastball,fastball,3.6,2.4,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=6e94035e-abf0-479f-82b1-981c2bf6ee93,https://baseballsavant.mlb.com/sporty-videos?playId=aba8d865-a279-4a51-b141-94c9e82868f8,2.4418
1460,fastball,fastball,4.8,3.6,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=cc383e59-619f-4657-a698-8109839f2424,https://baseballsavant.mlb.com/sporty-videos?playId=a09faaad-ad4a-4675-9809-364135bf34d0,2.4703
1488,fastball,breaking,7.2,7.2,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=800f824a-083e-41f2-9855-9afe90e46c24,https://baseballsavant.mlb.com/sporty-videos?playId=e963811c-140d-42ff-8f50-1f4795935aa0,2.79292
1384,fastball,fastball,3.6,3.6,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=6fbe6433-a074-42ca-892e-97ecefb5a416,https://baseballsavant.mlb.com/sporty-videos?playId=150068b7-ac14-42a6-bea5-c2c87c5885b7,2.8015
1307,fastball,fastball,3.6,4.8,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=1be5d6ea-d796-4b0c-aa76-011e8476de26,https://baseballsavant.mlb.com/sporty-videos?playId=b2b2d682-82cd-4d29-adbf-a44ea96df510,2.82057
1576,fastball,fastball,4.8,3.6,gid_2019_06_19_clemlb_texmlb_1,gid_2019_06_19_clemlb_texmlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=6b2fb015-210c-49e5-8235-e440cc4983e2,https://baseballsavant.mlb.com/sporty-videos?playId=150068b7-ac14-42a6-bea5-c2c87c5885b7,2.82814


Angel Hernandez


,simple_pitch,close_simple_pitch,break_length,close_break_length,game_id,close_game,url,close_url,distance_to_strike
1666,fastball,fastball,4.8,4.8,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=efb40e55-57a1-4447-abae-095581d88d0d,https://baseballsavant.mlb.com/sporty-videos?playId=3fc6c02f-e20d-4425-9086-1026bb87ccd9,1.76873
1688,fastball,fastball,4.8,3.6,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=fee3e813-66eb-45db-885c-93d0fc44b08c,https://baseballsavant.mlb.com/sporty-videos?playId=47bea139-f978-4b2b-a223-9790b0a5cd44,1.78539
1669,changeup,fastball,7.2,7.2,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=7276eaf0-26b4-4cbf-908e-ed1eaa1f2271,https://baseballsavant.mlb.com/sporty-videos?playId=211f66a7-bb04-46e1-9ebc-8a95716e1ad1,2.29399
1675,breaking,breaking,8.4,7.2,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=fbdbcc1b-9050-41bd-9233-f10bc1f6cea5,https://baseballsavant.mlb.com/sporty-videos?playId=beb64d87-2eb2-4c56-9656-32e9a45ba5eb,2.68991
1692,fastball,fastball,3.6,3.6,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=1429cf1b-c990-44f0-be44-2efb896913bf,https://baseballsavant.mlb.com/sporty-videos?playId=212fb63a-d258-4691-b3ea-4f02a3111288,3.12243
1620,fastball,fastball,3.6,4.8,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=24e9c4b8-b0e9-4df6-9519-c0d918f3a294,https://baseballsavant.mlb.com/sporty-videos?playId=35ffdc2f-a922-4b54-84f2-783cac774659,3.37289
1707,changeup,fastball,4.8,3.6,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=a7b6f5b5-84bf-4e75-9dee-dadd2eff845c,https://baseballsavant.mlb.com/sporty-videos?playId=47bea139-f978-4b2b-a223-9790b0a5cd44,3.77576
1826,fastball,fastball,4.8,3.6,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=299604fb-bec4-485c-82fb-6a97a47f821c,https://baseballsavant.mlb.com/sporty-videos?playId=f63b5dad-86fd-4378-b341-fb4b7fe8d037,3.79479
1633,fastball,changeup,4.8,6.0,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=a64c1bcc-2579-4b36-bff0-651a6fe6a001,https://baseballsavant.mlb.com/sporty-videos?playId=dc7d8665-5ffd-4c45-8dcb-ee268034634d,3.79605
1792,fastball,fastball,6.0,4.8,gid_2019_06_19_colmlb_arimlb_1,gid_2019_06_19_colmlb_arimlb_1,https://baseballsavant.mlb.com/sporty-videos?playId=5b5b5298-a45a-485e-ab82-8177a116a36b,https://baseballsavant.mlb.com/sporty-videos?playId=f7d6cf6e-0c5f-44fe-bc8a-de717a93a4dd,4.02114
